In [1]:
!pip install transformers
!pip install pandas
!pip install gradio
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## mODEL

In [2]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import gradio as gr
# Assuming 'df' is your DataFrame
# You can replace this with the actual name of your DataFrame

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# CONNECTION

In [3]:
import pinecone      

pinecone.init(      
	api_key='1b7c40a9-2a02-4f0c-b096-2e5d34ce2366',      
	environment='gcp-starter'      
)      
index = pinecone.Index('bigbasket')

## lOAD CSV

In [4]:
df = pd.read_csv('bigBasketProducts.csv')
df = df.drop('rating', axis=1)
# Drop all rows with null values in a particular column
column_with_nulls = 'description'
df = df.dropna(subset=[column_with_nulls])
df = df.dropna(subset=['brand'])
df = df.dropna(subset=['product'])

In [14]:
def vedant(query):
    # Tokenize input sentence and convert to tensor
    inputs = tokenizer(query, return_tensors="pt")
    # Forward pass through the BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the output embeddings from the last layer
    query_embedding = outputs.last_hidden_state.mean(dim=1)
    res = index.query(
      vector=query_embedding[0].tolist(),
      top_k=1,
      include_values=True
    )
    matches = res['matches']
    print("These are the top recommended products:")
    for match in matches:
        print(match)
        id = match['id']
        print(id)
        product = df.iloc[int(id)]
        # print(product)
    return product
    

In [15]:
df['description'][16725]

'Aigner No. 1 is a woody, dense, leathery scent that is softened by fresh accords that enhance the overall sense of allure. Cedar Leaves, Bergamont and Pepper capture the high notes whilst the powerful middle notes of Ginger, Cinnamon and Geranium add texture and depth. The whole infusion is given character and strength with bold Vetiver, Oud, Sandalwood and Musk.'

In [16]:
iface = gr.Interface(fn=vedant, inputs="text", outputs="text")
iface.launch()
# vedant('oily')

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


These are the top recommended products:
{'id': '16654',
 'score': 0.550538,
 'values': [-0.123550758,
            -0.0542153567,
            0.242303282,
            0.0218956862,
            0.12342333,
            0.0983636677,
            0.0541854911,
            0.163153544,
            0.239835769,
            -0.161341667,
            -0.100948676,
            -0.112345167,
            0.0319556184,
            0.264299721,
            0.144448519,
            0.268270463,
            0.187477455,
            0.0988857448,
            -0.0170946959,
            0.189978957,
            0.10895706,
            -0.25136736,
            0.14544408,
            0.147074088,
            0.611547947,
            0.121125199,
            0.121514626,
            0.0299191847,
            -0.172173843,
            0.096080035,
            0.340867162,
            0.264831275,
            -0.0566158108,
            -0.254201472,
            0.15229328,
            -0.0456099398,
        